In [ ]:
import duckdb as db
agg = db.sql(
"""
SELECT Ticker, count(*) as cnt
FROM 'localdatastore/year/2000-01-01_*.parquet'
group by Ticker
order by cnt desc
""")

daily = db.sql(
"""
SELECT *
FROM 'localdatastore/year/2000-01-01_*.parquet'
""")


meta = db.sql(
"""
select *, "ASX code" || '.AX' as yf_code
from './ASX_Listed_Companies_30-01-2025_02-54-26_AEDT.csv'
"""
)

group_stat = db.sql(
"""
select count(*) as cnt, "GICs industry group"
from meta
group by "GICs industry group"
order by cnt desc
""")


db.sql(
"""
with l1 as (
    select 
        d.*
        , m."GICs industry group" 
        , 1-d.Close/lag(d.Close) over (partition by d.Ticker order by d.Date) as close_pct_chng 
    from daily d
    left join meta m on d.Ticker = m.yf_code
)
select *
    , median(close_pct_chng) over (partition by d."GICs industry group" , d.Date) as industry_median_change
    , var_sample(close_pct_chng) over (partition by d."GICs industry group" , d.Date) as industry_chnge_var
from l1 d
where year(Date) > 2020
"""
).to_parquet("analysis_set.parquet")



In [52]:
group_stat

┌───────┬────────────────────────────────────────────────┐
│  cnt  │              GICs industry group               │
│ int64 │                    varchar                     │
├───────┼────────────────────────────────────────────────┤
│   797 │ Materials                                      │
│   124 │ Energy                                         │
│   113 │ Software & Services                            │
│   104 │ Financial Services                             │
│    84 │ Pharmaceuticals, Biotechnology & Life Sciences │
│    83 │ Not Applic                                     │
│    83 │ Health Care Equipment & Services               │
│    80 │ Capital Goods                                  │
│    53 │ Commercial & Professional Services             │
│    48 │ Consumer Services                              │
│     · │     ·                                          │
│     · │     ·                                          │
│     · │     ·                                         

In [17]:
import pandas as pd
df = pd.read_csv("./ASX_Listed_Companies_30-01-2025_02-54-26_AEDT.csv")
# print(df)
df['Market Cap'] = df['Market Cap'].apply(lambda v: None if v in ("SUSPENDED", "--") else int(v))
df200 = df.sort_values(by="Market Cap", ascending=False)#.iloc[:200]

x = 1835
display(df200.iloc[x:])


,ASX code,Company name,GICs industry group,Listing date,Market Cap
951,JAY,JAYRIDE GROUP LIMITED,Transportation,29/01/2018,477158.0
82,AGH,ALTHEA GROUP HOLDINGS LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",21/09/2018,NaN
90,AHI,ADVANCED HEALTH INTELLIGENCE LTD,Software & Services,17/08/2015,NaN
110,AKP,AUDIO PIXELS HOLDINGS LIMITED,Consumer Durables & Apparel,24/12/2004,NaN
132,AML,AEON METALS LIMITED.,Materials,14/06/2007,NaN
...,...,...,...,...,...
1870,WFL,WELLFULLY LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",29/05/2000,NaN
1882,WKT,WALKABOUT RESOURCES LTD,Materials,07/11/2006,NaN
1909,X64,TEN SIXTY FOUR LIMITED,Materials,23/12/2003,NaN
1917,XST,XSTATE RESOURCES LIMITED,Energy,14/01/1988,NaN


In [20]:
display(db.sql(
"""
select *
from df 
where Ticker = 'CLB.AX'
""").df())

,level_0,Ticker,Adj Close,Capital Gains,Close,Dividends,High,Low,Open,Stock Splits,Volume
0,2025-01-22 10:00:00+11:00,CLB.AX,NaN,0.0,0.18,0.0,0.18,0.18,0.18,0.0,3569.0
